In [ ]:
import sys

COLAB = "google.colab" in sys.modules

In [ ]:
import os
from shutil import copyfileobj
import zipfile

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from pathlib import Path
from urllib.request import urlopen

print(f"TF version: {tf.__version__}")

In [ ]:
def archive(dir: Path):
    with zipfile.ZipFile(f"{dir}.zip", "w", zipfile.ZIP_STORED) as zip_file:
        for entry in dir.rglob("*"):
            zip_file.write(entry, entry.relative_to(dir))


def unarchive(file: Path):
    with zipfile.ZipFile(file, "r") as zip_file:
        zip_file.extractall(file.with_suffix(""))

In [ ]:
def download_file(url, dataset_file_path):
    path = Path(dataset_file_path)
    os.makedirs(path.parent, exist_ok=True)
    if not path.exists():
        print(f"Downloading {path}")
        with urlopen(url) as fsrc, open(path, "wb") as fdst:
            copyfileobj(fsrc, fdst)
    else:
        print(f"File {path} exists")

In [ ]:
tmp_path = "tmp/image_classification"
dataset_path = f"{tmp_path}/dataset"
batch_size = 32
image_width = 256
image_height = 256
image_size = (image_width, image_height)
image_size_3 = image_size + (3,)
seed = 123
class_count = 20

In [ ]:
download_file("https://huggingface.co/datasets/Matthijs/snacks/resolve/main/images.zip", f"{dataset_path}.zip")

In [ ]:
unarchive(Path(f"tmp/image_classification/dataset.zip"))

In [ ]:
train_dataset = tf.keras.utils.image_dataset_from_directory(
    directory=f"{dataset_path}/data/train",
    label_mode="categorical",
    batch_size=batch_size,
    image_size=image_size,
    shuffle=True,
    seed=seed,
    interpolation="bicubic",
)
validation_dataset = tf.keras.utils.image_dataset_from_directory(
    directory=f"{dataset_path}/data/val",
    label_mode="categorical",
    batch_size=batch_size,
    image_size=image_size,
    shuffle=False,
    seed=seed,
    interpolation="bicubic",
)
assert train_dataset.class_names == validation_dataset.class_names
assert len(train_dataset.class_names) == class_count
train_dataset=train_dataset.cache().prefetch(10000).shuffle(10000)
validation_dataset=validation_dataset.cache().prefetch(10000)

In [ ]:
# model = tf.keras.Sequential([
#     tf.keras.layers.Input(image_size_3),

#     tf.keras.layers.RandomFlip(mode="horizontal"),
#     tf.keras.layers.RandomRotation(0.05),
#     tf.keras.layers.RandomZoom(0.15),
#     tf.keras.layers.RandomTranslation(0.1, 0.1),
#     tf.keras.layers.RandomContrast(0.15),
#     tf.keras.layers.RandomBrightness(0.1, value_range=[0.0, 255.0]),
#     tf.keras.layers.Layer(name="augmentation_end"),

#     tf.keras.layers.Rescaling(1 / 255.0),

#     # Pixel noise
#     tf.keras.layers.Dropout(0.1, noise_shape=image_size + (1,), name=f"dropout_xy_1"),
#     # Horizontal line noise
#     tf.keras.layers.Dropout(0.05, noise_shape=(image_height, 1, 1), name=f"dropout_x_1"),
#     # Vertical line noise
#     tf.keras.layers.Dropout(0.05, noise_shape=(1, image_width, 1), name=f"dropout_y_1"),

#     tf.keras.layers.Conv2D(32, (3, 3), activation="gelu", name=f"conv2d_1"),
#     tf.keras.layers.MaxPooling2D((2, 2), name=f"maxpooling2d_1"),
#     tf.keras.layers.Dropout(0.05, name=f"dropout_1"),
#     tf.keras.layers.BatchNormalization(name=f"batch_norm_1"),

#     tf.keras.layers.Conv2D(64, (3, 3), activation="gelu", name=f"conv2d_2"),
#     tf.keras.layers.MaxPooling2D((2, 2), name=f"maxpooling2d_2"),
#     tf.keras.layers.Dropout(0.05, name=f"dropout_2"),
#     tf.keras.layers.BatchNormalization(name=f"batch_norm_2"),

#     tf.keras.layers.Conv2D(64, (3, 3), activation="gelu", name=f"conv2d_3"),
#     tf.keras.layers.MaxPooling2D((2, 2), name=f"maxpooling2d_3"),
#     tf.keras.layers.Dropout(0.05, name=f"dropout_3"),
#     tf.keras.layers.BatchNormalization(name=f"batch_norm_3"),

#     tf.keras.layers.Conv2D(128, (3, 3), activation="gelu", name=f"conv2d_4"),
#     tf.keras.layers.MaxPooling2D((2, 2), name=f"maxpooling2d_4"),
#     tf.keras.layers.Dropout(0.05, name=f"dropout_4"),
#     tf.keras.layers.BatchNormalization(name=f"batch_norm_4"),

#     tf.keras.layers.Conv2D(128, (3, 3), activation="gelu", name=f"conv2d_5"),
#     tf.keras.layers.MaxPooling2D((2, 2), name=f"maxpooling2d_5"),
#     tf.keras.layers.Dropout(0.05, name=f"dropout_5"),
#     tf.keras.layers.BatchNormalization(name=f"batch_norm_5"),

#     tf.keras.layers.Conv2D(256, (3, 3), activation="gelu", name=f"conv2d_6"),
#     tf.keras.layers.MaxPooling2D((2, 2), name=f"maxpooling2d_6"),
#     tf.keras.layers.Dropout(0.05, name=f"dropout_6"),
#     tf.keras.layers.BatchNormalization(name=f"batch_norm_6"),

#     tf.keras.layers.Flatten(name=f"flatten"),
#     tf.keras.layers.Dropout(0.1, name=f"dropout_7"),
#     tf.keras.layers.Dense(64, activation="gelu", name=f"dense_1"),
#     tf.keras.layers.Dense(class_count, activation="softmax", name=f"output"),
# ])

In [ ]:
def create_model():
    input = tf.keras.layers.Input(image_size_3)
    layer = input

    layer = tf.keras.layers.RandomFlip(mode="horizontal")(layer)
    layer = tf.keras.layers.RandomRotation(0.05)(layer)
    layer = tf.keras.layers.RandomZoom(0.15)(layer)
    layer = tf.keras.layers.RandomTranslation(0.1, 0.1)(layer)
    layer = tf.keras.layers.RandomContrast(0.15)(layer)
    layer = tf.keras.layers.RandomBrightness(0.1, value_range=[0.0, 255.0])(layer)
    layer = tf.keras.layers.Layer(name="augmentation_end")(layer)

    layer = tf.keras.layers.Rescaling(1 / 255.0)(layer)

    # # Pixel noise
    # layer = tf.keras.layers.Dropout(0.1, noise_shape=image_size + (1,), name=f"dropout_xy_1")(layer)
    # # Horizontal line noise
    # layer = tf.keras.layers.Dropout(0.1, noise_shape=(image_height, 1, 1), name=f"dropout_x_1")(layer)
    # # Vertical line noise
    # layer = tf.keras.layers.Dropout(0.1, noise_shape=(1, image_width, 1), name=f"dropout_y_1")(layer)

    layer = tf.keras.layers.Conv2D(64, (3, 3), padding="same", activation="relu", name=f"conv2d_1")(layer)
    layer = tf.keras.layers.MaxPooling2D((2, 2), name=f"maxpooling2d_1")(layer)
    # layer = tf.keras.layers.Dropout(0.1, name=f"dropout_1")(layer)
    # layer = tf.keras.layers.BatchNormalization(name=f"batch_norm_1")(layer)
    # previous_block = layer

    layer = tf.keras.layers.Conv2D(64, (3, 3), padding="same", activation="relu", name=f"conv2d_2")(layer)
    layer = tf.keras.layers.MaxPooling2D((2, 2), name=f"maxpooling2d_2")(layer)
    # layer = tf.keras.layers.Dropout(0.1, name=f"dropout_2")(layer)
    # layer = tf.keras.layers.BatchNormalization(name=f"batch_norm_2")(layer)
    # layer = tf.keras.layers.Add()([layer, tf.keras.layers.AveragePooling2D((2, 2))(previous_block)])
    # previous_block = layer

    layer = tf.keras.layers.Conv2D(128, (3, 3), padding="same", activation="relu", name=f"conv2d_3")(layer)
    layer = tf.keras.layers.MaxPooling2D((2, 2), name=f"maxpooling2d_3")(layer)
    layer = tf.keras.layers.Dropout(0.1, name=f"dropout_3")(layer)
    # layer = tf.keras.layers.BatchNormalization(name=f"batch_norm_3")(layer)
    # previous_block = tf.keras.layers.AveragePooling2D((2, 2))(previous_block)
    # previous_block = tf.repeat(previous_block, repeats=2, axis=-1)
    # # layer = tf.keras.layers.Add()([layer, previous_block])
    # previous_block = layer

    layer = tf.keras.layers.Conv2D(128, (3, 3), padding="same", activation="relu", name=f"conv2d_4")(layer)
    layer = tf.keras.layers.MaxPooling2D((2, 2), name=f"maxpooling2d_4")(layer)
    layer = tf.keras.layers.Dropout(0.05, name=f"dropout_4")(layer)
    # layer = tf.keras.layers.BatchNormalization(name=f"batch_norm_4")(layer)
    # layer = tf.keras.layers.Add()([layer, tf.keras.layers.AveragePooling2D((2, 2))(previous_block)])
    # previous_block = layer

    # layer = tf.keras.layers.Conv2D(256, (3, 3), padding="same", activation="gelu", name=f"conv2d_5")(layer)
    # layer = tf.keras.layers.MaxPooling2D((2, 2), name=f"maxpooling2d_5")(layer)
    # layer = tf.keras.layers.Dropout(0.15, name=f"dropout_5")(layer)
    # layer = tf.keras.layers.BatchNormalization(name=f"batch_norm_5")(layer)
    # previous_block = tf.keras.layers.AveragePooling2D((2, 2))(previous_block)
    # previous_block = tf.repeat(previous_block, repeats=2, axis=-1)
    # # layer = tf.keras.layers.Add()([layer, previous_block])
    # previous_block = layer

    # layer = tf.keras.layers.Conv2D(256, (3, 3), padding="same", activation="gelu", name=f"conv2d_6")(layer)
    # layer = tf.keras.layers.MaxPooling2D((2, 2), name=f"maxpooling2d_6")(layer)
    # layer = tf.keras.layers.Dropout(0.15, name=f"dropout_6")(layer)
    # layer = tf.keras.layers.BatchNormalization(name=f"batch_norm_6")(layer)
    # # layer = tf.keras.layers.Add()([layer, tf.keras.layers.AveragePooling2D((2, 2))(previous_block)])
    # previous_block = layer

    layer = tf.keras.layers.Flatten(name=f"flatten")(layer)
    layer = tf.keras.layers.Dropout(0.15, name=f"dropout_7")(layer)
    layer = tf.keras.layers.Dense(256, activation="relu", name=f"dense_1")(layer)
    layer = tf.keras.layers.Dense(class_count, activation="softmax", name=f"output")(layer)

    return tf.keras.Model(inputs=input, outputs=layer)


model = create_model()

In [ ]:
from tensorflow.keras import layers

model = tf.keras.Sequential([
  tf.keras.layers.Input(image_size_3),

  tf.keras.layers.RandomFlip(mode="horizontal"),
  tf.keras.layers.RandomRotation(0.05),
  tf.keras.layers.RandomZoom(0.15),
  tf.keras.layers.RandomTranslation(0.1, 0.1),
  tf.keras.layers.RandomContrast(0.15),
  tf.keras.layers.RandomBrightness(0.1, value_range=[0.0, 255.0]),
  tf.keras.layers.Layer(name="augmentation_end"),

  tf.keras.layers.Rescaling(1 / 255.0),

  # Pixel noise
  # layers.Dropout(0.05, noise_shape=image_size + (1,), name=f"dropout_xy_1"),
  # # Horizontal line noise
  # layers.Dropout(0.1, noise_shape=(image_height, 1, 1), name=f"dropout_x_1"),
  # # Vertical line noise
  # layers.Dropout(0.1, noise_shape=(1, image_width, 1), name=f"dropout_y_1"),

  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  # layers.Dropout(0.05),
  # layers.LayerNormalization(),
  # layers.BatchNormalization(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  # layers.Dropout(0.05),
  # layers.LayerNormalization(),
  # layers.BatchNormalization(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  # layers.Dropout(0.05),
  # layers.LayerNormalization(),
  # layers.BatchNormalization(),
  layers.Flatten(),
  # layers.Dropout(0.2),
  layers.Dense(128, activation='relu'),
  layers.Dense(class_count, activation="softmax", name=f"output")
])

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.AdamW(learning_rate=1e-3),
    loss="categorical_crossentropy",
    metrics=["acc"]
)

In [ ]:
# checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
#     filepath=f"tmp/checkpoints",
#     save_weights_only=False,
#     monitor=f"val_acc",
#     mode="max",
#     save_best_only=True,
# )
reduce_lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor=f"val_acc",
    mode="max",
)

In [ ]:
if not COLAB:
    !aws s3 cp "s3://.../tf/tf-exam/image-classification/model.h5" "$tmp_path/model.h5"

In [ ]:
model.load_weights(f"{tmp_path}/model.h5")

In [ ]:
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=20,
    callbacks=[reduce_lr_callback],
)

In [ ]:
acc = history.history["acc"]
val_acc = history.history["val_acc"]

loss = history.history["loss"]
val_loss = history.history["val_loss"]

epochs_range = range(len(acc))

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(epochs_range, acc, label="Training accuracy")
plt.plot(epochs_range, val_acc, label="Validation accuracy")
plt.legend(loc="lower left")

plt.subplot(2, 1, 2)
plt.plot(epochs_range, loss, label="Training Loss")
plt.plot(epochs_range, val_loss, label="Validation Loss")
plt.legend(loc="lower left")
plt.show()

In [ ]:
model.save("tmp/model.h5", save_format="h5")